In [1]:
from math import sqrt
import pandas as pd
from matplotlib import pyplot
from pathlib import Path
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import concatenate

Using TensorFlow backend.
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/alexellard/opt/anacond

In [36]:
S238_path = '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-238'
S239_path = '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-239'
S240_path = '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-240'
S249_path = '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-249'
S251_path = '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-251'

scenario_paths = [S238_path, S239_path, S240_path, S249_path, S251_path]

In [37]:
scenario_paths

['/Users/alexellard/Downloads/Hanoi_CMH/Scenario-238',
 '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-239',
 '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-240',
 '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-249',
 '/Users/alexellard/Downloads/Hanoi_CMH/Scenario-251']

In [33]:
demands_df = []
flows_df = []
leaks_df = []
pressures_df = []
for files in S250_path:
    timestamps = (S250_path+'/Timestamps.csv')
    demands_path = glob.glob(S250_path+'/Demands/*.csv')
    demands_path.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    flows_path = glob.glob(S250_path+'/Flows/*.csv')
    flows_path.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    leaks_path = glob.glob(S250_path+'/Leaks/*.csv')
    pressures_path = glob.glob(S250_path+'/Pressures/*.csv')
    pressures_path.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
timestamps_df = pd.read_csv(timestamps)

#Read demands
for files in demands_path:
    df = pd.read_csv(files)
    demands_df.append(df)
for i, df in enumerate(demands_df):
    demands_df[i] = pd.merge(df, timestamps_df, on='Index')
    demands_df[i].set_index('Timestamp', inplace=True)

#Read flows
for files in flows_path:
    df = pd.read_csv(files)
    flows_df.append(df)
for i, df in enumerate(flows_df):
    flows_df[i] = pd.merge(df, timestamps_df, on='Index')
    flows_df[i].set_index('Timestamp', inplace=True)
    
#Read Leaks
#for files in leaks_path:
#    df = pd.read_csv(files)
#    leaks_df.append(df)
#for i, df in enumerate(leaks_df):
#    demands_df[i] = pd.merge(df, timestamps_df, on='Index')
#    demands_df[i].set_index('Timestamp', inplace=True)
    
for files in pressures_path:
    df = pd.read_csv(files)
    pressures_df.append(df)
for i, df in enumerate(pressures_df):
    pressures_df[i] = pd.merge(df, timestamps_df, on='Index')
    pressures_df[i].set_index('Timestamp', inplace=True)

In [38]:
len(demands_df)

32

In [15]:
demands_merged_df = []
for i in range(len(demands_df)):
    new_df = timestamps_df.merge(demands_df[i], on='Index')
    demands_merged_df.append(new_df)
all_nodes_df = pd.concat(demands_merged_df, axis=1, join='inner')
all_nodes_df = all_nodes_df.T.drop_duplicates().T
all_nodes_df

,Index,Timestamp,Value,Value,Value,Value,Value,Value,Value,Value,...,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,...,183.6,140.4,28.8,140.4,54.0,54.0,61.2,50.4,18.0,133.2
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,...,162.0,118.8,25.2,111.6,46.8,50.4,57.6,46.8,18.0,115.2
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,...,147.6,104.4,25.2,122.4,43.2,39.6,43.2,43.2,14.4,104.4
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,...,122.4,100.8,25.2,100.8,39.6,36.0,43.2,36.0,10.8,86.4
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,...,111.6,100.8,21.6,86.4,36.0,32.4,39.6,36.0,10.8,79.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17516,2017-12-31 21:30:00,-5475.6,241.2,237.6,32.4,187.2,306.0,352.8,144.0,...,313.2,201.6,50.4,234.0,104.4,75.6,104.4,100.8,32.4,237.6
17516,17517,2017-12-31 22:00:00,-5252.4,248.4,216.0,32.4,180.0,284.4,313.2,144.0,...,277.2,212.4,43.2,230.4,86.4,79.2,104.4,90.0,28.8,237.6
17517,17518,2017-12-31 22:30:00,-4852.8,230.4,198.0,28.8,172.8,270.0,302.4,129.6,...,298.8,190.8,39.6,205.2,86.4,75.6,86.4,79.2,28.8,216.0
17518,17519,2017-12-31 23:00:00,-4446.0,216.0,187.2,25.2,158.4,234.0,291.6,129.6,...,234.0,165.6,43.2,183.6,72.0,68.4,82.8,75.6,28.8,172.8


In [16]:
new_column_names = ['Index'] + ['Timestamp'] + ['reservoir_demand'] + [f'node_{i}_demand' for i in range(1, 32)]

all_nodes_df.columns = new_column_names

In [17]:
all_nodes_df

,Index,Timestamp,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,...,node_22_demand,node_23_demand,node_24_demand,node_25_demand,node_26_demand,node_27_demand,node_28_demand,node_29_demand,node_30_demand,node_31_demand
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,...,183.6,140.4,28.8,140.4,54.0,54.0,61.2,50.4,18.0,133.2
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,...,162.0,118.8,25.2,111.6,46.8,50.4,57.6,46.8,18.0,115.2
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,...,147.6,104.4,25.2,122.4,43.2,39.6,43.2,43.2,14.4,104.4
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,...,122.4,100.8,25.2,100.8,39.6,36.0,43.2,36.0,10.8,86.4
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,...,111.6,100.8,21.6,86.4,36.0,32.4,39.6,36.0,10.8,79.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17516,2017-12-31 21:30:00,-5475.6,241.2,237.6,32.4,187.2,306.0,352.8,144.0,...,313.2,201.6,50.4,234.0,104.4,75.6,104.4,100.8,32.4,237.6
17516,17517,2017-12-31 22:00:00,-5252.4,248.4,216.0,32.4,180.0,284.4,313.2,144.0,...,277.2,212.4,43.2,230.4,86.4,79.2,104.4,90.0,28.8,237.6
17517,17518,2017-12-31 22:30:00,-4852.8,230.4,198.0,28.8,172.8,270.0,302.4,129.6,...,298.8,190.8,39.6,205.2,86.4,75.6,86.4,79.2,28.8,216.0
17518,17519,2017-12-31 23:00:00,-4446.0,216.0,187.2,25.2,158.4,234.0,291.6,129.6,...,234.0,165.6,43.2,183.6,72.0,68.4,82.8,75.6,28.8,172.8


In [18]:
demands_pressures_merged_df = []
for i in range(len(pressures_df)):
    new_df = all_nodes_df.merge(pressures_df[i], on='Index')
    demands_pressures_merged_df.append(new_df)
all_demands_pressures_df = pd.concat(demands_pressures_merged_df, axis=1, join='inner')
all_demands_pressures_df = all_demands_pressures_df.T.drop_duplicates().T
all_demands_pressures_df

,Index,Timestamp,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,...,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,...,67.674,67.567,67.418,67.301,67.253,67.391,67.224,67.223,67.244,67.328
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,...,68.11,68.027,67.91,67.821,67.77,67.875,67.741,67.741,67.76,67.836
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,...,68.481,68.411,68.311,68.229,68.2,68.304,68.198,68.194,68.206,68.256
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,...,68.743,68.686,68.608,68.543,68.515,68.598,68.511,68.511,68.522,68.565
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,...,68.876,68.826,68.759,68.706,68.674,68.753,68.678,68.676,68.685,68.722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17516,2017-12-31 21:30:00,-5475.6,241.2,237.6,32.4,187.2,306.0,352.8,144.0,...,64.143,63.872,63.466,63.175,63.054,63.423,62.912,62.892,62.953,63.199
17516,17517,2017-12-31 22:00:00,-5252.4,248.4,216.0,32.4,180.0,284.4,313.2,144.0,...,64.62,64.354,63.969,63.686,63.573,63.922,63.456,63.446,63.5,63.714
17517,17518,2017-12-31 22:30:00,-4852.8,230.4,198.0,28.8,172.8,270.0,302.4,129.6,...,65.334,65.117,64.803,64.571,64.472,64.763,64.411,64.397,64.434,64.595
17518,17519,2017-12-31 23:00:00,-4446.0,216.0,187.2,25.2,158.4,234.0,291.6,129.6,...,66.091,65.913,65.652,65.453,65.356,65.594,65.287,65.283,65.324,65.492


In [19]:
new_column_names_pressures_demands = ['Index'] + ['Timestamp'] + ['reservoir_demand'] + [f'node_{i}_demand' for i in range(1, 32)] + ['reservoir_demand'] + [f'node_{i}_pressure' for i in range(1, 32)]

all_demands_pressures_df.columns = new_column_names_pressures_demands

all_demands_pressures_df

,Index,Timestamp,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,...,node_22_pressure,node_23_pressure,node_24_pressure,node_25_pressure,node_26_pressure,node_27_pressure,node_28_pressure,node_29_pressure,node_30_pressure,node_31_pressure
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,...,67.674,67.567,67.418,67.301,67.253,67.391,67.224,67.223,67.244,67.328
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,...,68.11,68.027,67.91,67.821,67.77,67.875,67.741,67.741,67.76,67.836
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,...,68.481,68.411,68.311,68.229,68.2,68.304,68.198,68.194,68.206,68.256
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,...,68.743,68.686,68.608,68.543,68.515,68.598,68.511,68.511,68.522,68.565
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,...,68.876,68.826,68.759,68.706,68.674,68.753,68.678,68.676,68.685,68.722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17516,2017-12-31 21:30:00,-5475.6,241.2,237.6,32.4,187.2,306.0,352.8,144.0,...,64.143,63.872,63.466,63.175,63.054,63.423,62.912,62.892,62.953,63.199
17516,17517,2017-12-31 22:00:00,-5252.4,248.4,216.0,32.4,180.0,284.4,313.2,144.0,...,64.62,64.354,63.969,63.686,63.573,63.922,63.456,63.446,63.5,63.714
17517,17518,2017-12-31 22:30:00,-4852.8,230.4,198.0,28.8,172.8,270.0,302.4,129.6,...,65.334,65.117,64.803,64.571,64.472,64.763,64.411,64.397,64.434,64.595
17518,17519,2017-12-31 23:00:00,-4446.0,216.0,187.2,25.2,158.4,234.0,291.6,129.6,...,66.091,65.913,65.652,65.453,65.356,65.594,65.287,65.283,65.324,65.492


In [34]:
demands_pressures_flows_merged_df = []
for i in range(len(flows_df)):
    new_df = all_demands_pressures_df.merge(flows_df[i], on='Index')
    demands_pressures_flows_merged_df.append(new_df)
all_demands_pressures_flows_df = pd.concat(demands_pressures_flows_merged_df, axis=1, join='inner')
all_demands_pressures_flows_df = all_demands_pressures_flows_df.T.drop_duplicates().T
all_demands_pressures_flows_df

,Index,Timestamp,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,node_10_demand,node_11_demand,node_12_demand,node_13_demand,node_14_demand,node_15_demand,node_16_demand,node_17_demand,node_18_demand,node_19_demand,node_20_demand,node_21_demand,node_22_demand,node_23_demand,node_24_demand,node_25_demand,node_26_demand,node_27_demand,node_28_demand,node_29_demand,node_30_demand,node_31_demand,reservoir_demand,node_1_pressure,node_2_pressure,node_3_pressure,node_4_pressure,node_5_pressure,node_6_pressure,node_7_pressure,node_8_pressure,node_9_pressure,node_10_pressure,node_11_pressure,node_12_pressure,node_13_pressure,node_14_pressure,node_15_pressure,node_16_pressure,node_17_pressure,node_18_pressure,node_19_pressure,node_20_pressure,node_21_pressure,node_22_pressure,node_23_pressure,node_24_pressure,node_25_pressure,node_26_pressure,node_27_pressure,node_28_pressure,node_29_pressure,node_30_pressure,node_31_pressure,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,90.0,90.0,75.6,104.4,154.8,93.6,39.6,54.0,140.4,234.0,10.8,241.2,140.4,79.2,183.6,140.4,28.8,140.4,54.0,54.0,61.2,50.4,18.0,133.2,0.0,69.855,68.175,68.093,67.938,67.747,67.718,67.622,67.579,67.547,67.491,67.46,67.333,67.268,67.252,67.251,67.327,67.595,67.926,67.879,67.521,67.277,67.674,67.567,67.418,67.301,67.253,67.391,67.224,67.223,67.244,67.328,3355.2,3189.6,1278.0,1260.0,1134.0,972.0,748.8,658.8,568.8,338.4,262.8,140.4,46.8,3.6,46.8,-187.2,-421.2,-432.0,1328.4,219.6,867.6,565.2,428.4,-198.0,-57.6,-3.6,118.8,64.8,3.6,-50.4,68.4,198.0
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,79.2,68.4,61.2,97.2,162.0,79.2,39.6,46.8,140.4,223.2,7.2,223.2,129.6,64.8,162.0,118.8,25.2,111.6,46.8,50.4,57.6,46.8,18.0,115.2,0.0,69.882,68.51,68.442,68.315,68.155,68.131,68.05,68.014,67.987,67.935,67.904,67.766,67.767,67.751,67.748,67.778,68.001,68.291,68.272,67.992,67.825,68.11,68.027,67.91,67.821,67.77,67.875,67.741,67.741,67.76,67.836,2998.8,2862.0,1152.0,1134.0,1029.6,874.8,684.0,597.6,518.4,324.0,262.8,122.4,46.8,7.2,28.8,-169.2,-392.4,-399.6,1180.8,194.4,763.2,493.2,378.0,-172.8,-57.6,-14.4,104.4,57.6,-0.0,-46.8,64.8,180.0
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,79.2,61.2,54.0,82.8,126.0,72.0,32.4,39.6,126.0,183.6,7.2,187.2,115.2,64.8,147.6,104.4,25.2,122.4,43.2,39.6,43.2,43.2,14.4,104.4,0.0,69.906,68.809,68.757,68.658,68.536,68.517,68.453,68.425,68.405,68.37,68.349,68.264,68.216,68.203,68.2,68.233,68.42,68.642,68.615,68.364,68.195,68.481,68.411,68.311,68.229,68.2,68.304,68.198,68.194,68.206,68.256,2656.8,2534.4,1000.8,986.4,889.2,766.8,604.8,518.4,439.2,262.8,208.8,115.2,43.2,7.2,28.8,-154.8,-338.4,-345.6,1058.4,183.6,687.6,450.0,345.6,-165.6,-43.2,-0.0,90.0,50.4,3.6,-36.0,50.4,154.8
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,68.4,64.8,50.4,93.6,118.8,61.2,32.4,39.6,108.0,162.0,7.2,187.2,115.2,57.6,122.4,100.8,25.2,100.8,39.6,36.0,43.2,36.0,10.8,86.4,0.0,69.921,69.012,68.967,68.883,68.779,68.763,68.707,68.682,68.663,68.628,68.607,68.53,68.522,68.512,68.511,68.546,68.7,68.878,68.848,68.625,68.499,68.743,68.686,68.608,68.543,68.515,68.598,68.511,68.511,68.522,68.565,2408.4,2289.6,918.0,907.2,813.6,709.2,558.0,493.2,424.8,262.8,212.4,97.2,36.0,3.6,32.4,-140.4,-302.4,-306.0,964.8,172.8,604.8,403.2,302.4,-144.0,-43.2,-3.6,82.8,43.2,0.0,-36.0,46.8,133.2
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,68.4,57.6,46.8,79.2,122.4,54.0,28.8,36.0,115.2,151.2,7.2,165.6,93.6,54.0,111.6,100.8,21.6,86.4,36.0,32.4,39.6,36.0,10.8,79.2,0.0,69.929,69.105,69.063,68.983,68.883,68.867,68.817,68.795,68.779,68.747,68.728,68.65,68.667,68.658,68.658,68.675,68.821,68.983,68.968,68.793,68.67,68.876

In [35]:
new_column_names_pressures_demands_flows = ['Index'] + ['Timestamp'] + ['reservoir_demand'] + [f'node_{i}_demand' for i in range(1, 32)] + ['reservoir_pressure'] + [f'node_{i}_pressure' for i in range(1, 32)] + [f'link_{i}_flow' for i in range(1, 33)]

all_demands_pressures_flows_df.columns = new_column_names_pressures_demands_flows

pd.set_option('display.max_columns', None)
all_demands_pressures_flows_df

,Index,Timestamp,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,node_10_demand,node_11_demand,node_12_demand,node_13_demand,node_14_demand,node_15_demand,node_16_demand,node_17_demand,node_18_demand,node_19_demand,node_20_demand,node_21_demand,node_22_demand,node_23_demand,node_24_demand,node_25_demand,node_26_demand,node_27_demand,node_28_demand,node_29_demand,node_30_demand,node_31_demand,reservoir_pressure,node_1_pressure,node_2_pressure,node_3_pressure,node_4_pressure,node_5_pressure,node_6_pressure,node_7_pressure,node_8_pressure,node_9_pressure,node_10_pressure,node_11_pressure,node_12_pressure,node_13_pressure,node_14_pressure,node_15_pressure,node_16_pressure,node_17_pressure,node_18_pressure,node_19_pressure,node_20_pressure,node_21_pressure,node_22_pressure,node_23_pressure,node_24_pressure,node_25_pressure,node_26_pressure,node_27_pressure,node_28_pressure,node_29_pressure,node_30_pressure,node_31_pressure,link_1_flow,link_2_flow,link_3_flow,link_4_flow,link_5_flow,link_6_flow,link_7_flow,link_8_flow,link_9_flow,link_10_flow,link_11_flow,link_12_flow,link_13_flow,link_14_flow,link_15_flow,link_16_flow,link_17_flow,link_18_flow,link_19_flow,link_20_flow,link_21_flow,link_22_flow,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
0,1,2017-01-01 00:00:00,-3355.2,162.0,154.8,18.0,126.0,162.0,223.2,90.0,90.0,90.0,75.6,104.4,154.8,93.6,39.6,54.0,140.4,234.0,10.8,241.2,140.4,79.2,183.6,140.4,28.8,140.4,54.0,54.0,61.2,50.4,18.0,133.2,0.0,69.855,68.175,68.093,67.938,67.747,67.718,67.622,67.579,67.547,67.491,67.46,67.333,67.268,67.252,67.251,67.327,67.595,67.926,67.879,67.521,67.277,67.674,67.567,67.418,67.301,67.253,67.391,67.224,67.223,67.244,67.328,3355.2,3189.6,1278.0,1260.0,1134.0,972.0,748.8,658.8,568.8,338.4,262.8,140.4,46.8,3.6,46.8,-187.2,-421.2,-432.0,1328.4,219.6,867.6,565.2,428.4,-198.0,-57.6,-3.6,118.8,64.8,3.6,-50.4,68.4,198.0
1,2,2017-01-01 00:30:00,-2998.8,140.4,129.6,18.0,100.8,158.4,190.8,86.4,79.2,68.4,61.2,97.2,162.0,79.2,39.6,46.8,140.4,223.2,7.2,223.2,129.6,64.8,162.0,118.8,25.2,111.6,46.8,50.4,57.6,46.8,18.0,115.2,0.0,69.882,68.51,68.442,68.315,68.155,68.131,68.05,68.014,67.987,67.935,67.904,67.766,67.767,67.751,67.748,67.778,68.001,68.291,68.272,67.992,67.825,68.11,68.027,67.91,67.821,67.77,67.875,67.741,67.741,67.76,67.836,2998.8,2862.0,1152.0,1134.0,1029.6,874.8,684.0,597.6,518.4,324.0,262.8,122.4,46.8,7.2,28.8,-169.2,-392.4,-399.6,1180.8,194.4,763.2,493.2,378.0,-172.8,-57.6,-14.4,104.4,57.6,-0.0,-46.8,64.8,180.0
2,3,2017-01-01 01:00:00,-2656.8,122.4,129.6,14.4,97.2,122.4,165.6,86.4,79.2,61.2,54.0,82.8,126.0,72.0,32.4,39.6,126.0,183.6,7.2,187.2,115.2,64.8,147.6,104.4,25.2,122.4,43.2,39.6,43.2,43.2,14.4,104.4,0.0,69.906,68.809,68.757,68.658,68.536,68.517,68.453,68.425,68.405,68.37,68.349,68.264,68.216,68.203,68.2,68.233,68.42,68.642,68.615,68.364,68.195,68.481,68.411,68.311,68.229,68.2,68.304,68.198,68.194,68.206,68.256,2656.8,2534.4,1000.8,986.4,889.2,766.8,604.8,518.4,439.2,262.8,208.8,115.2,43.2,7.2,28.8,-154.8,-338.4,-345.6,1058.4,183.6,687.6,450.0,345.6,-165.6,-43.2,-0.0,90.0,50.4,3.6,-36.0,50.4,154.8
3,4,2017-01-01 01:30:00,-2408.4,115.2,100.8,14.4,93.6,104.4,151.2,64.8,68.4,64.8,50.4,93.6,118.8,61.2,32.4,39.6,108.0,162.0,7.2,187.2,115.2,57.6,122.4,100.8,25.2,100.8,39.6,36.0,43.2,36.0,10.8,86.4,0.0,69.921,69.012,68.967,68.883,68.779,68.763,68.707,68.682,68.663,68.628,68.607,68.53,68.522,68.512,68.511,68.546,68.7,68.878,68.848,68.625,68.499,68.743,68.686,68.608,68.543,68.515,68.598,68.511,68.511,68.522,68.565,2408.4,2289.6,918.0,907.2,813.6,709.2,558.0,493.2,424.8,262.8,212.4,97.2,36.0,3.6,32.4,-140.4,-302.4,-306.0,964.8,172.8,604.8,403.2,302.4,-144.0,-43.2,-3.6,82.8,43.2,0.0,-36.0,46.8,133.2
4,5,2017-01-01 02:00:00,-2278.8,108.0,108.0,10.8,79.2,108.0,165.6,68.4,68.4,57.6,46.8,79.2,122.4,54.0,28.8,36.0,115.2,151.2,7.2,165.6,